In [3]:
import json
import pandas as pd
import datetime

In [20]:
#Concatenation des dataframes
dfConcat = pd.read_csv('Data/Solde_Patrimoine.csv')

dfConcat['Date_last_sync'] = pd.to_datetime(dfConcat['Date_last_sync']).dt.tz_localize(None)
dfConcat['Date_extract'] = pd.to_datetime(dfConcat['Date_extract']).dt.tz_localize(None)

In [22]:
dfConcat.dtypes

Date_last_sync    datetime64[ns]
Date_extract      datetime64[ns]
Agregat1_ID               object
Organisme                 object
Produit                   object
Solde                    float64
Week_number                int64
dtype: object

In [21]:
dfConcat

,Date_last_sync,Date_extract,Agregat1_ID,Organisme,Produit,Solde,Week_number
0,2023-04-21 10:34:43.677,2023-04-21 14:01:55.368,1.1,Allianz,Assurance Vie,33391.480230,16
1,2023-04-21 11:57:24.409,2023-04-21 14:01:55.368,2.4,Amundi,PEE / PERCO,20933.062490,16
2,2023-04-21 00:32:38.274,2023-04-21 14:01:55.368,7.1.3,Linxea,Assurance Vie Avenir,19101.573657,16
3,2023-04-21 07:36:05.571,2023-04-21 14:01:55.368,7.1.2,Linxea,Assurance Vie Spirit 2,2158.682589,16
4,2023-04-18 03:45:03.035,2023-04-21 14:01:55.368,5.4,Groupama,PEE / PERCO,3780.696666,16
...,...,...,...,...,...,...,...
139,2023-06-01 13:43:02.443,2023-06-02 01:00:10.504,5.4,Groupama,PEE / PERCO,6536.463488,22
140,2023-06-01 12:25:18.506,2023-06-02 01:00:10.504,6.1,HSBC,Assurance Vie,19470.794018,22
141,2023-06-01 21:08:16.261,2023-06-02 01:00:10.504,3.2,BourseDirect,PEA,154951.772600,22
142,2023-06-01 19:20:36.562,2023-06-02 01:00:10.504,4.3,Boursorama,PEA-PMA,4289.703600,22


In [4]:
dfConcat['Date_extract'].dt.isocalendar().week

0     16
1     16
2     16
3     16
4     16
      ..
76    21
77    21
78    21
79    21
80    21
Name: week, Length: 81, dtype: UInt32

In [5]:
dfComptes = pd.read_excel('Param/Description_Comptes.xlsx')

In [8]:
# Ouvrir le fichier UC en mode lecture
with open("Data/BalanceHeritageUC.json", "r") as f:
    # Charger le contenu du fichier en tant qu'objet JSON
    json_data_UC = json.load(f)

BalancePatrimoine = {'ID':[],"Date_last_sync" :[], "Date_extract" : [],"Nom":[], "Type":[] ,"Solde" : []}

# Afficher l'objet JSON UC sous forme de dictionnaire Python
for i in range(len(json_data_UC["result"])):
    
    BalancePatrimoine["ID"].append(json_data_UC["result"][i]["id"])

    BalancePatrimoine["Date_last_sync"].append(json_data_UC["result"][i]["last_sync"])

    BalancePatrimoine["Date_extract"].append(datetime.datetime.now())

    BalancePatrimoine["Nom"].append(json_data_UC["result"][i]["name"])

    BalancePatrimoine["Type"].append("UC")

    BalancePatrimoine["Solde"].append(json_data_UC["result"][i]["balance"])
    

# Convertir le dictionnaire en DataFrame
df = pd.DataFrame(BalancePatrimoine)

In [9]:
dfAgregat = pd.merge(df, dfComptes, on='ID', how='left')

In [14]:
dfGrouped = dfAgregat.groupby('Agregat1').agg(Date_last_sync = ('Date_last_sync', 'first'), Date_extract = ('Date_extract', 'first'), Agregat1_ID = ('Agregat1_ID', 'first') ,Organisme =('Organisme', 'first'), Produit =('Produit', 'first'), Solde = ('Solde', 'sum'))


In [16]:
dfGrouped['Week_number'] = dfGrouped['Date_extract'].dt.isocalendar().week

In [17]:
dfGrouped

,Date_last_sync,Date_extract,Agregat1_ID,Organisme,Produit,Solde,Week_number
Agregat1,,,,,,,
Allianz,2023-05-28T11:59:32.669Z,2023-05-30 22:31:05.774571,1.1,Allianz,Assurance Vie,33426.019091,22
Allianz PEE / PERCO,2023-05-28T16:08:52.971Z,2023-05-30 22:31:05.774558,2.4,Amundi,PEE / PERCO,20524.976557,22
Assurance Vie Avenir,2023-05-28T13:45:57.140Z,2023-05-30 22:31:05.774587,7.1.3,Linxea,Assurance Vie Avenir,19011.735471,22
Assurance Vie Spirit 2,2023-05-28T16:16:39.076Z,2023-05-30 22:31:05.774585,7.1.2,Linxea,Assurance Vie Spirit 2,2164.344155,22
Groupama PEE / PERCO,2023-05-28T15:20:39.037Z,2023-05-30 22:31:05.774574,5.4,Groupama,PEE / PERCO,6464.014000,22
HSBC,2023-05-28T13:18:26.815Z,2023-05-30 22:31:05.774593,6.1,HSBC,Assurance Vie,19407.494889,22
PEA,2023-05-28T10:20:38.764Z,2023-05-30 22:31:05.774567,3.2,BourseDirect,PEA,155519.718000,22
PEA-PMA,2023-05-28T11:10:26.918Z,2023-05-30 22:31:05.774581,4.3,Boursorama,PEA-PMA,4239.826800,22
Yomoni,2023-05-28T11:21:27.283Z,2023-05-30 22:31:05.774583,8.1,Yomoni,Assurance Vie,9375.492241,22


## Rajout dans l'historique numéro semaine et ID aggregat1

In [16]:
dfHisto = pd.read_excel('Data/Solde_Patrimoine.xlsx')

dfHisto['Date_last_sync'] = pd.to_datetime(dfHisto['Date_last_sync']).dt.tz_localize(None)
dfHisto['Date_extract'] = pd.to_datetime(dfHisto['Date_extract']).dt.tz_localize(None)

In [18]:
dfHisto.dtypes

Date_last_sync    datetime64[ns]
Date_extract      datetime64[ns]
Agregat1_ID               object
Organisme                 object
Produit                   object
Solde                    float64
Week_number                int64
dtype: object

In [19]:
dfHisto.to_csv('Data/Solde_Patrimoine.csv', sep=',', encoding='utf-8', index=False)